In [1]:
#|export
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter()
from nbdev.export import nb_export

# Set page configuration
st.set_page_config(layout="wide")

In [2]:
#|export
# generic libraries
import pandas as pd
import numpy as np
import time
import uuid

# pdf processing functions
import fitz
import pdf_utils

# NLP models
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [3]:
#|export
# create pipeline for environmental classifier
name = "ESGBERT/EnvironmentalBERT-environmental"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
esg_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

C:\Users\ambar\anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
#|export
# create pipeline for netzero-reduction classifier
name = "ClimatePolicyRadar/national-climate-targets"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
cpr_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, function_to_apply="sigmoid")

In [5]:
#|export
# function to encapsulate all preprocessing steps
def pdf_to_sentences(path, max_char, min_words=5):
    text = pdf_utils.true_pdf2text(path=path) # check if true PDF
    
    if len(text) == 0:
        true_pdf = False
        
        # OCR processing time based on no. of pages
        doc = fitz.open(stream=path, filetype="pdf")
        est_time = round(5.0*len(doc)/60) 
        l = ["⏳Running preprocessing... Estimated time remaining:", str(est_time), "mins"]
        placeholder = app.empty()
        placeholder.write(" ".join(l))

        # run OCR extraction
        output_dir = r'D:\Work\Climate NLP\web apps\pdf_images'
        text = pdf_utils.scan_pdf2text(path=path, output_dir=output_dir)
        
        placeholder.empty()
    
    text = text[:max_char] # spacy has a 1M char limit
    sents = pdf_utils.text2sents(text=text, min_words=min_words)
    
    return sents

In [6]:
#|export
# function to run a 2-stage classification
def classify(path, cutoff_score, max_char=1000000):
    # run preprocessing steps
    sents = pdf_to_sentences(path, max_char)
    
    # NLP processing time per sentence based on test runs
    est_time = round(0.1*len(sents)/60) 
    l = ["⏳Running classification... Estimated time remaining:", str(est_time), "mins"]
    placeholder = app.empty()
    placeholder.write(" ".join(l))

    # run stage-1 classifier
    s1_class = esg_pipe(sents, padding=True, truncation=True)
    
    # create data frame with texts labeled as environmental
    env_labels = [x['label'] for x in s1_class]
    df = pd.DataFrame({"text": sents, "env_label": env_labels})
    df = df[df['env_label'] == "environmental"]
    
    # print count of total vs env sentences
    app.subheader("Document Summary", divider="blue")
    l = [str(len(sents)), "sentences in document.", str(len(df)), "classified as Environmental."]
    app.write(" ".join(l))
    
    # run stage-2 classifier
    sents_s1 = df.text.to_list()
    s2_class = cpr_pipe(sents_s1, padding=True, truncation=True)
    
    # merge stage-2 labels into df
    df['label'] = [x['label'] for x in s2_class]
    df['score'] = [x['score'] for x in s2_class]
    
    # set labels to None where score is below cutoff
    df.loc[df['score'] < cutoff_score, 'label'] = 'None'
    
    placeholder.empty()
    
    return(df)

In [7]:
#|export
# generate content for About page
def setup_about_page(file, img1, img2):
    # demo data download
    with open(file, "rb") as pdf_file:
        demo_data = pdf_file.read()
    
    txt_demo = '''👋Welcome to Scetti's Netzero-Reduction Detector tool!
    If you do not have a policy document at hand, you can try out the app with a sample document.'''
    ack.markdown(txt_demo)
    ack.download_button(label="Download demo document",
                        data=demo_data,
                        file_name="demo_policy_document.pdf",
                        mime='application/octet-stream')    
    
    # methodology notes
    ack.subheader("Methodology", divider="blue")
    ack.markdown("Each document is analyzed through a 3-stage process described below")
    ack.image(img1)

    # preprocessing flowchart
    ack.markdown(":one: The extraction of sentences from a PDF document goes through a series of steps")
    ack.image(img2)
    
    # acknowledgements
    ack_file = r'D:\Work\Climate NLP\web apps\acknowledgements.txt'
    with open(ack_file, "r") as file:
        ack_txt = file.readlines()
    ack.markdown(ack_txt[1]) # ESGBERT/environmental acknowledgement
    ack.markdown(ack_txt[3]) # ClimatePolicyRadar acknowledgement
    
    # contact info
    txt_fb = '''📧Tell us about your experience with the Netzero-Reduction Detector! Please send your 
    feedback and suggestions to <a href='mailto:ambar@scetti.org'> ambar@scetti.org</a>'''
    ack.markdown(txt_fb, unsafe_allow_html=True)

In [8]:
#|export
# function to keep count of app runs
def update_run_count():
    run_count_file = r'D:\Work\Climate NLP\web apps\run_count.txt'

    # read current value of run_count from file
    with open(run_count_file, "r") as file:
        run_count = int(file.read())

    # increment the count and write it back to the file
    run_count += 1
    with open(run_count_file, "w") as file:
        file.write(str(run_count))

    # st.write(f"This Streamlit app has been run {run_count} times.")
    return run_count    

In [9]:
#|export
st.title("Netzero-Reduction Detector")

# user input widgets (form inside sidebar)
with st.sidebar:
    with st.form("user_inputs"):
        uploaded_file = st.file_uploader(label="Upload a PDF document",
                                         type=[".pdf"])
        
        annotate_pdf = st.checkbox("Highlight target text within document", 
                                   value=True,
                                   help="Not supported for scanned PDFs")
        
        cutoff_score = st.slider(label="Minimum confidence threshold (recommended 0.50)",
                                 help="Setting this too low can produce false positives",
                                 min_value=0.0, 
                                 max_value=1.0, 
                                 value=0.5, 
                                 step=0.05)
        
        submitted = st.form_submit_button("Analyze document")

# create separate tabs for analysis and acknowledgements
app, ack = st.tabs(["📈Analysis", "📢About"])

# setup About page
x = setup_about_page(file = r'D:\Work\Climate NLP\web apps\demo_policy_document.pdf',
                     img1 = r'D:\Work\Climate NLP\web apps\nlp_process_flow.png',
                     img2 = r'D:\Work\Climate NLP\web apps\preprocessing_flowchart.png')

# description for Analysis tab
txt_intro = '''As a researcher working on climate policy, have you tried searching for measurable goals or targets relating to GHG emission reduction? 
This app uses AI-based Natural Language Processing to detect occurences of Netzero, Reduction, and Other targets in your climate policy document.

🌟:green[June 2024 update: scanned/image PDF documents are now supported!]'''
app.markdown(txt_intro)

if uploaded_file and submitted:
    start_time = time.time()
    x = update_run_count()
    true_pdf = True # boolean flag for true vs scanned PDF
    
    # run 2-stage classification on a PDF (set max_char=1000 for testing)
    path = uploaded_file.read()
    df = classify(path=path, cutoff_score=cutoff_score)
    
    if df is not None:
        # plot stage-2 label freq
        chart_data = df.groupby('label').size().reset_index(name='sentences')
        app.bar_chart(chart_data, 
                     x='label', 
                     y='sentences')

        # print sentences not classified as 'None'
        app.subheader("Text classified as Netzero, Reduction, or Other target", divider="blue")
        app.markdown(":information_source: Double-click on text to expand. Hover on top right corner of table for more options.")
        df1 = df[df.label!='None'].drop('env_label', axis=1)
        app.dataframe(df1, 
                     hide_index=True, 
                     use_container_width=True)

        # display PDF with target text highlighted (not supported for scanned PDF)
        if annotate_pdf and true_pdf:
            target_text = df1.text.to_list()
            pages = pdf_utils.annotate_target_text(path=path, target_text=target_text)
            for page in pages:
                app.image(page, use_column_width=True)

        # print completion status
        app.write("Analysis completed! 👍")
        time_taken = round((time.time() - start_time)/60, 1)
        l = ["Time taken:", str(time_taken), "mins"]
        app.write(" ".join(l))

        # CSV file for user download
        file_name = uploaded_file.name.replace('pdf', 'csv')
        csv = df.to_csv(index=False).encode('utf-8')
        app.download_button(label="Download CSV output", 
                            data=csv, 
                            file_name=file_name)

# Netzero-Reduction Detector

2024-06-19 12:31:50.916 
  command:

    streamlit run C:\Users\ambar\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Checkbox(value=True, description='Highlight target text within document', indent=False)

In [10]:
# exports this notebook to a .py file with streamlit statements added
nb_export("NetZero_Detector.ipynb", lib_path="./", name="NetZero_Detector")